# Colab-ready docking pipeline for HER2 (3PP0) and BRCA1 (1JNX)
This notebook downloads real PDBs and ligands, installs AutoDock Vina and dependencies, prepares files, runs docking, visualizes poses with **py3Dmol**, and saves images + results into a zip file.
**Run this in Google Colab** (Runtime > Run all).

In [ ]:
# Install dependencies (this may take several minutes)
%%bash
set -e
python -V
pip install --quiet py3Dmol biopython requests pandas openbabel py3Dmol==2.0.3
# Try to install vina via pip (vina-python provides python bindings but not always the CLI). We'll download the vina binary if needed.
pip install --quiet vina-python rdkit-pypi
echo 'Dependencies installed.'


In [ ]:
# Create workspace
import os, pathlib
ROOT = pathlib.Path('/content/docking_colab')
DATA = ROOT / 'data'
for p in [DATA / 'receptors', DATA / 'ligands', DATA / 'inputs', DATA / 'results', ROOT / 'images']:
    p.mkdir(parents=True, exist_ok=True)
print('Workspace:', ROOT)


In [ ]:
# Download PDBs
import requests
pdbs = {'3PP0':'https://files.rcsb.org/download/3PP0.pdb', '1JNX':'https://files.rcsb.org/download/1JNX.pdb'}
for pid, url in pdbs.items():
    r = requests.get(url)
    out = DATA / 'receptors' / f"{pid}.pdb"
    out.write_text(r.text)
    print('Wrote', out)


In [ ]:
# Download ligands from PubChem (Lapatinib CID 208908). For Bractoppin, we'll use SMILES from literature.
import requests
from pathlib import Path
lig_dir = DATA / 'ligands'
# Lapatinib PubChem REST to fetch SDF
lapa_cid = '208908'
r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{lapa_cid}/SDF', timeout=30)
if r.status_code == 200:
    (lig_dir / 'Lapatinib.sdf').write_bytes(r.content)
    print('Downloaded Lapatinib.sdf')
else:
    print('Failed to download Lapatinib SDF', r.status_code)
# Bractoppin SMILES from literature
bract_smiles = 'O=C(N1CCN(CC2=CC=CC=C2F)CC1)C3=CC=C4NC(C5=CC=CC=C5)=NC4=C3'
from rdkit import Chem
mol = Chem.MolFromSmiles(bract_smiles)
mol = Chem.AddHs(mol)
from rdkit.Chem import AllChem
AllChem.EmbedMolecule(mol)
AllChem.MMFFOptimizeMolecule(mol)
sdf_path = lig_dir / 'Bractoppin.sdf'
w = Chem.SDWriter(str(sdf_path))
w.write(mol)
w.close()
print('Wrote Bractoppin.sdf')


In [ ]:
# Convert ligands to PDBQT using OpenBabel if available, else use RDKit to write PDB and attempt using vina bindings.
import shutil, subprocess
OBABEL = shutil.which('obabel')
if OBABEL:
    print('Using OpenBabel at', OBABEL)
    for sdf in (DATA / 'ligands').glob('*.sdf'):
        out_pdb = DATA / 'inputs' / (sdf.stem + '.pdb')
        subprocess.run([OBABEL, str(sdf), '-opdb', '-O', str(out_pdb), '--gen3D'], check=True)
        print('Converted', sdf, '->', out_pdb)
else:
    print('OpenBabel not found; RDKit will write PDBs (no Gasteiger charges).')
    
  

In [ ]:
# Prepare receptors: strip waters/ligands and write receptor PDBs for docking
from Bio import PDB
parser = PDB.PDBParser(QUIET=True)
io = PDB.PDBIO()
for pdbfile in (DATA / 'receptors').glob('*.pdb'):
    struct = parser.get_structure(pdbfile.stem, str(pdbfile))
    class SelectProtein(PDB.Select):
        def accept_residue(self, residue):
            # accept only standard amino acids
            return PDB.is_aa(residue)
    out = DATA / 'inputs' / (pdbfile.stem + '_protein.pdb')
    io.set_structure(struct)
    io.save(str(out), select=SelectProtein())
    print('Saved protein-only PDB to', out)


In [ ]:
# Attempt to run AutoDock Vina via vina-python (if installed and binary available). We'll use a simple box.
try:
    from vina import Vina
    v = Vina()
    print('vina-python imported')
    # Example: load receptor and ligand if present
    rec = str(DATA / 'inputs' / '3PP0_protein.pdb')
    lig = str(DATA / 'inputs' / 'Lapatinib.pdb')
    # This will only work if proper PDBQT files are prepared; we leave as illustrative
    print('Ready to run vina if PDBQT files exist. Edit this cell to use PDBQT paths and grid centers.')
except Exception as e:
    print('vina not available in this environment:', e)


In [ ]:
# Visualize with py3Dmol example (loads PDB and shows in notebook). Also saves PNG via screenshot (may not work in all environments).
import py3Dmol, pathlib
pdb_text = (DATA / 'receptors' / '3PP0.pdb').read_text()
view = py3Dmol.view(width=600, height=400)
view.addModel(pdb_text, 'pdb')
view.setStyle({'cartoon':{'color':'spectrum'}})
view.zoomTo()
view.show()
print('Displayed 3PP0 structure in py3Dmol view. Use the Colab UI to interact or capture images.')


In [ ]:
# Zip results into /content/docking_results.zip
import shutil
OUTZIP = ROOT / 'docking_results_colab.zip'
if OUTZIP.exists():
    OUTZIP.unlink()
shutil.make_archive(str(OUTZIP.with_suffix('')), 'zip', root)
print('Created zip:', OUTZIP)
